### 문장 & 단어 예측
- 데이터셋
    * 문장의 단어를 N-gram 기반으로 구성해서 생성
        - 예) "오늘은 즐거운 피자 먹는 금요일 입니다."
        - '오늘은 즐거운', '오늘은 즐거운 피자', '오늘은 즐거운 피자 먹는',
        - '오늘은 즐거운 피자 먹는 금요일', '오늘은 즐거운 피자 먹는 금요일 입니다.'
    * N-gram 기반으로 생성된 데이터셋의 마지막 요소 ==> 레이블

- 정확한 텍스트 데이터 준비가 중요
    * 맞춤법, 띄어쓰기 잘 되어있는 데이터 준비
### 데이터준비

In [14]:
songdata = '''The temperature is falling, and so am I
Drifting through the snow just to feel like I'm getting by
I wish that I could light a fire and find my paradise
I wonder, will you be there when I find the light?
Winter is coming, it's cold outside
This frozen world froze you on my mind
Will you be mine? Will you be mine?
Will you be mine and keep my heart warm every night?
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine
I feel the cold kissing my skin
And all I think about is you again
'Cause the way you got me is a whole year thing
But right now especially
I wonder, will you be there when I find the light?
Winter is coming, it's cold outside
This frozen world froze you on my mind
Will you be mine? Will you be mine?
Will you be mine and keep my heart warm every night?
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine
The seasons change just like a wave
All we can do is hang on now
But today, let's run away
Our frozen legs can never slow us down
Winter is coming, it's cold outside
This frozen world froze you on my mind
Will you be mine? Will you be mine?
Will you be mine and keep my heart warm every night?
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine
Can I put all of my love in you and I?
Will you be mine? Yeah, be mine'''

#### 텍스트 데이터 기반 단어 사전 생성
- keras Tokenizer() 사용하여 구두점 삭제, 공백 기준으로 분할
- 단어 생성

In [15]:
import tensorflow as tf


In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower = True)


In [17]:
songdatalist = songdata.split("\n")

In [18]:
tokenizer.fit_on_texts(songdatalist)

In [44]:
total_words= len(tokenizer.word_index)+1 #voca num

In [71]:
word_index=tokenizer.word_index

In [21]:
seqdata1 = tokenizer.texts_to_sequences(songdatalist)

In [26]:
len(seqdata1)

37

#### 숫자화된 데이터를 다시 N-gram 방식으로 새롭게 생성

In [24]:
seqdata2 =[]
for line in seqdata1:
    for idx in range(2,len(line)+1):
        seqdata2.append(line[:idx])

In [25]:
print(seqdata2)

[[10, 44], [10, 44, 11], [10, 44, 11, 45], [10, 44, 11, 45, 7], [10, 44, 11, 45, 7, 46], [10, 44, 11, 45, 7, 46, 47], [10, 44, 11, 45, 7, 46, 47, 3], [48, 49], [48, 49, 10], [48, 49, 10, 50], [48, 49, 10, 50, 36], [48, 49, 10, 50, 36, 51], [48, 49, 10, 50, 36, 51, 37], [48, 49, 10, 50, 36, 51, 37, 38], [48, 49, 10, 50, 36, 51, 37, 38, 52], [48, 49, 10, 50, 36, 51, 37, 38, 52, 53], [48, 49, 10, 50, 36, 51, 37, 38, 52, 53, 54], [3, 55], [3, 55, 56], [3, 55, 56, 3], [3, 55, 56, 3, 57], [3, 55, 56, 3, 57, 20], [3, 55, 56, 3, 57, 20, 21], [3, 55, 56, 3, 57, 20, 21, 58], [3, 55, 56, 3, 57, 20, 21, 58, 7], [3, 55, 56, 3, 57, 20, 21, 58, 7, 22], [3, 55, 56, 3, 57, 20, 21, 58, 7, 22, 6], [3, 55, 56, 3, 57, 20, 21, 58, 7, 22, 6, 59], [3, 39], [3, 39, 5], [3, 39, 5, 1], [3, 39, 5, 1, 2], [3, 39, 5, 1, 2, 40], [3, 39, 5, 1, 2, 40, 41], [3, 39, 5, 1, 2, 40, 41, 3], [3, 39, 5, 1, 2, 40, 41, 3, 22], [3, 39, 5, 1, 2, 40, 41, 3, 22, 10], [3, 39, 5, 1, 2, 40, 41, 3, 22, 10, 20], [23, 11], [23, 11, 24], 

In [29]:
maxlen = max([len(i) for i in seqdata2])

In [34]:
padded_seq =  tf.keras.preprocessing.sequence.pad_sequences(seqdata2, maxlen = maxlen,padding='pre')
padded_seq

array([[ 0,  0,  0, ...,  0, 10, 44],
       [ 0,  0,  0, ..., 10, 44, 11],
       [ 0,  0,  0, ..., 44, 11, 45],
       ...,
       [ 0,  0,  0, ...,  2,  4, 16],
       [ 0,  0,  0, ...,  4, 16,  2],
       [ 0,  0,  0, ..., 16,  2,  4]])

### [2]학습용 데이터 준비

In [46]:
data = padded_seq[:, :-1]
labels = padded_seq[:,-1]

#### 보통 label은 엔코딩을 함

In [48]:
labels = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [104]:
class make_new_sentence(tf.keras.Model):
    def __init__(self):
        super(make_new_sentence, self).__init__()
        self.Embeddinglayer = tf.keras.Sequential([
            tf.keras.layers.Embedding(total_words, 8)
        ])
        self.lstmlayer = tf.keras.Sequential([
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(11, return_sequences = True)),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(11))
        ])
        self.outlayer = tf.keras.Sequential([
            tf.keras.layers.Dense(total_words, activation='softmax')           
        ])
    def call(self, x):
        x = self.Embeddinglayer(x)
        x = self.lstmlayer(x)
        x = self.outlayer(x)
        return x

In [106]:
model = make_new_sentence()
model.build(input_shape = (None,50))

In [107]:
model.summary()

Model: "make_new_sentence_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_14 (Sequential)  (None, None, 8)           720       
                                                                 
 sequential_15 (Sequential)  (None, 22)                4752      
                                                                 
 sequential_16 (Sequential)  (None, 90)                2070      
                                                                 
Total params: 7,542
Trainable params: 7,542
Non-trainable params: 0
_________________________________________________________________


In [108]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [109]:
history = model.fit(data, labels, epochs=1500, verbose=1)

Epoch 1/1500
9/9 [==============================] - 6s 6ms/step - loss: 4.4961 - accuracy: 0.0463 
Epoch 2/1500
9/9 [==============================] - 0s 6ms/step - loss: 4.4822 - accuracy: 0.0605
Epoch 3/1500
9/9 [==============================] - 0s 5ms/step - loss: 4.4628 - accuracy: 0.0605
Epoch 4/1500
9/9 [==============================] - 0s 6ms/step - loss: 4.4267 - accuracy: 0.0605
Epoch 5/1500
9/9 [==============================] - 0s 6ms/step - loss: 4.3573 - accuracy: 0.0605
Epoch 6/1500
9/9 [==============================] - 0s 10ms/step - loss: 4.2355 - accuracy: 0.0605
Epoch 7/1500
9/9 [==============================] - 0s 8ms/step - loss: 4.0873 - accuracy: 0.0605
Epoch 8/1500
9/9 [==============================] - 0s 7ms/step - loss: 3.9652 - accuracy: 0.0605
Epoch 9/1500
9/9 [==============================] - 0s 6ms/step - loss: 3.9056 - accuracy: 0.0747
Epoch 10/1500
9/9 [==============================] - 0s 6ms/step - loss: 3.8713 - accuracy: 0.0996
Epoch 11/1500
9/9

In [110]:
seed_text = 'Winter is'
token_list = tokenizer.texts_to_sequences([seed_text])[0]

In [90]:
token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list],maxlen = 11, padding = 'pre')

In [97]:
import numpy as np
predicted = np.argmax(model.predict(token_list),axis=1)

In [98]:
for word, index in tokenizer.word_index.items():
    if index == predicted:
        print(word)
        break

coming


In [99]:
seed_text = 'Winter is'
next_words = 10
for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list],maxlen = 11, padding = 'pre')
    predicted = np.argmax(model.predict(token_list),axis=1)
    outputword = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            outputword = word
            break
    seed_text += " "+outputword
print(seed_text)   

Winter is coming it's cold outside outside you again am i i'm


it's
